In [1]:
import mysql.connector
from faker import Faker
import re
import unicodedata
from datetime import datetime, timedelta, date
import random
from decimal import Decimal, ROUND_HALF_UP

connection = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

cursor = connection.cursor()

fake = Faker("pl_PL")

In [2]:
tabele = [
    "Oceny", 
    "Rezerwacje", 
    "Realizowane_wycieczki", 
    "Rodzaje_wycieczek", 
    "Miejsce_przylotu", 
    "Miejsce_wylotu", 
    "Pracownicy", 
    "Stanowisko", 
    "Koszty_organizacji", 
    "Transakcje_finansowe", 
    "Klienci",
    "Metody_platnosci"
]
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

for tabela in tabele:
    cursor.execute(f"TRUNCATE TABLE {tabela};")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

connection.commit()

In [3]:
# Wypełnianie tabeli Miejsce_wylotu

insert_wylot_query = """
INSERT INTO Miejsce_wylotu (id_wylot, Miasto) 
VALUES (%s, %s);
"""

miasta_wylotu = ['Warszawa', 'Kraków', 'Gdańsk', 'Wrocław', 'Poznań']
for id_wylot, Miasto in enumerate(miasta_wylotu, start=1):  
    cursor.execute(insert_wylot_query, (id_wylot, Miasto))

connection.commit()

In [4]:
# Wypełnianie tabeli Miejsce_przylotu

insert_przylot_query = """
INSERT INTO Miejsce_przylotu (id_przylot, Miasto) 
VALUES (%s, %s);
"""

miasta_przylotu = ['Cairns', 'Alice Springs', 'Sydney', 'Gold Coast', 'Adelaide', 'Melbourne', 'Brisbane']
for id_przylot, Miasto in enumerate(miasta_przylotu, start=1):  # Jeden wpis na miasto
    cursor.execute(insert_przylot_query, (id_przylot, Miasto))

connection.commit()

In [5]:
# Wypełnianie tabeli Stanowisko

insert_stanowisko_query = """
INSERT INTO Stanowisko (Kategoria, Koszt_stanowiska) 
VALUES (%s, %s);
"""

kategorie = [
    {"kategoria": "Manager", "koszt": 1000.00},
    {"kategoria": "Programista", "koszt": 750.00},
    {"kategoria": "Księgowy", "koszt": 750.00},
    {"kategoria": "Specjalista ds. marketingu", "koszt": 650.00},
    {"kategoria": "Konsultant ds. obsługi klienta", "koszt": 500.00}
]

for kategoria in kategorie:
    cursor.execute(insert_stanowisko_query, (kategoria["kategoria"], kategoria["koszt"]))

connection.commit()

In [6]:
# Wypełnianie tabeli Klienci

def usun_polskie(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'
    )

domeny = ["gmail.com", "yahoo.com", "outlook.com", "wp.pl", "interia.pl", "onet.pl"]
def generuj_email(imie, nazwisko):
    domena = random.choice(domeny)
    return f"{usun_polskie(imie.lower())}.{usun_polskie(nazwisko.lower())}@{domena}"

def data_urodzenia():
    grupy = [
        (18, 20, 0.1),  # 10% osób w wieku 18-20 lat
        (21, 50, 0.65),  # 65% osób w wieku 21-50 lat
        (51, 80, 0.25),  # 25% osób w wieku 51-80 lat
    ]
    group = random.choices(grupy, weights=[x[2] for x in grupy], k=1)[0]
    birth_date = fake.date_of_birth(minimum_age=group[0], maximum_age=group[1])
    return birth_date.strftime('%Y-%m-%d')

for _ in range(2700):
    Imie = fake.first_name()
    Nazwisko = fake.last_name()
    Email = generuj_email(Imie, Nazwisko)
    Numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    Ulica = fake.street_name()
    Numer_lokalu = fake.building_number()
    Miasto = fake.city()
    Kod_pocztowy = fake.postcode()
    Numer_telefonu_bliskiej_osoby = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    Data_urodzenia = data_urodzenia()
    
    cursor.execute("""
        INSERT INTO Klienci 
        (Imie, Nazwisko, Numer_telefonu, Email, Ulica, Numer_lokalu, Miasto, Kod_pocztowy, Dane_kontaktowe_bliskich, Data_urodzenia)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (Imie, Nazwisko, Numer_telefonu, Email, Ulica, Numer_lokalu, Miasto, Kod_pocztowy, Numer_telefonu_bliskiej_osoby, Data_urodzenia))

connection.commit()

In [7]:
# Wypełnianie tabeli Pracownicy

def generuj_pensje(id_stanowisko):
    if id_stanowisko == 1:  # Manager
        return random.randint(12000, 15000)
    elif id_stanowisko in [2, 3]:  # Programista, Księgowy
        return random.randint(10000, 12000)
    elif id_stanowisko == 4:  # Specjalista ds. marketingu
        return random.randint(8000, 10000)
    else:  # Konsultant ds. obsługi klienta
        return random.randint(5000, 8000)

def generuj_date_urodzenia(id_stanowisko):
    if id_stanowisko == 1:  # Manager
        return fake.date_of_birth(minimum_age=30, maximum_age=45)
    elif id_stanowisko in [2, 3, 4]:  # Programista, Księgowy, Specjalista ds. marketingu
        return fake.date_of_birth(minimum_age=25, maximum_age=45)
    else:  # Konsultant ds. obsługi klienta
        return fake.date_of_birth(minimum_age=21, maximum_age=45)

def losuj_date_zatrudnienia(id_stanowisko):
    if id_stanowisko in [1, 2]:  # Manager, Programista
        start_date = date(2022, 1, 1)
        end_date = date(2022, 1, 5)
    else:  # Pozostałe stanowiska
        start_date = date(2022, 1, 5)
        end_date = date(2022, 1, 14)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

kolejnosc_stanowisk = [1, 2, 3, 4, 5, 5, 5] 

for id_stanowisko in kolejnosc_stanowisk:
    imie = fake.first_name()
    nazwisko = fake.last_name()
    email = generuj_email(imie, nazwisko)
    numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    ulica = fake.street_name()
    numer_lokalu = fake.building_number()
    miasto = fake.city()
    kod_pocztowy = fake.postcode()
    pensja = generuj_pensje(id_stanowisko)
    data_urodzenia = generuj_date_urodzenia(id_stanowisko)
    data_zatrudnienia = losuj_date_zatrudnienia(id_stanowisko)

    cursor.execute("""
        INSERT INTO Pracownicy (Imie, Nazwisko, id_stanowisko, Numer_telefonu, Email, Ulica, Numer_lokalu, Miasto, Kod_pocztowy, Pensja, Data_urodzenia, Data_zatrudnienia)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (imie, nazwisko, id_stanowisko, numer_telefonu, email, ulica, numer_lokalu, miasto, kod_pocztowy, pensja, data_urodzenia, data_zatrudnienia))

connection.commit()


In [8]:
# Wypełnanie tabeli Rodzaje_wycieczek

insert_rodzaje_query = """
INSERT INTO Rodzaje_wycieczek (id_rodzaj, Nazwa, Opis, id_wylot, id_przylot) 
VALUES (%s, %s, %s, %s, %s);
"""

wycieczki = [
    (1, "Wielka Rafa Koralowa i podwodne przygody", 
     "Odkryj podwodny świat jednego z największych cudów natury! Czeka Cię rejs na Wielką Rafę Koralową, gdzie będziesz nurkować lub snorkelować wśród kolorowych ryb. W programie znajdziesz także wieczorny rejs z kolacją pod gwiazdami oraz wycieczkę do tropikalnego lasu Daintree. Dla bardziej odważnych przewidziano opcję nurkowania w klatce z rekinami (za dodatkową opłatą).", 
     1, 1),
    
    (2, "Pustynna Odyseja", 
     "Wybierz się w podróż przez australijskie pustkowia, zaczynając od Alice Springs – serca Outbacku. Odwiedzisz tajemnicze formacje skalne, takie jak Devils Marbles, i spróbujesz lokalnych atrakcji, takich jak rzucanie bumerangiem. Następnie czeka Cię eksploracja Uluru i okolicznych parków narodowych, w tym przejażdżka wielbłądami przez pustynne wydmy. Wieczory spędzisz w wygodnych namiotach lub w hotelach w buszu, gdzie spróbujesz słynnych australijskich lamingtonów.", 
     2, 2),
    
    (3, "Survival w australijskim buszu", 
     "Gotowy na wyzwanie? Survival w buszu to wyprawa tylko dla odważnych! W sercu pustynnej Australii, w okolicach Alice Springs, nauczysz się, jak przetrwać w trudnych warunkach: znajdziesz wodę, zbudujesz schronienie z gałęzi i liści oraz rozpalisz ogień bez zapałek. Profesjonalni przewodnicy pokażą Ci, jak unikać niebezpiecznych zwierząt i rozpoznawać jadalne rośliny. Na koniec wyprawy zmierzysz się z “testem przetrwania” – spędzeniem jednej nocy samemu w buszu. Nie martw się – instruktorzy będą czuwać!", 
     3, 2),
    
    (4, "Wielkomiejska przygoda w Sydney", 
     "Tydzień w Sydney to połączenie wielkomiejskiego życia, kultury i relaksu. Odwiedzisz najsłynniejsze miejsca, takie jak Opera w Sydney, most Harbour Bridge, czy przepiękna plaża Bondi. Wieczorami zanurzysz się w nocnym życiu Sydney, odwiedzając lokalne bary, gdzie spróbujesz rzemieślniczych piw i prawdziwych australijskich koktajli.", 
     4, 3),
    
    (5, "Surfing i plażowanie", 
     "Marzysz o złotych plażach i falach idealnych do surfingu? Na Gold Coast nauczysz się surfować z pomocą profesjonalnych instruktorów. Czeka Cię także wyprawa na Fraser Island – największą na świecie piaszczystą wyspę. Dni wypełnione surfingiem i opalaniem zakończą się wieczornymi ogniskami na plaży z widokiem na rozgwieżdżone niebo.", 
     2, 4),
    
    (6, "Adelajda i Winne Doliny", 
     "Tygodniowa wyprawa do Adelajdy to doskonałe połączenie miejskich atrakcji i wycieczek za miasto. Zwiedzisz eleganckie centrum Adelajdy, odwiedzając galerie sztuki, muzea i słynny Central Market, gdzie spróbujesz lokalnych smakołyków. Następnie wyruszysz do Doliny Barossa, jednej z najsłynniejszych australijskich krain winiarskich. Czeka Cię degustacja wina w kilku winiarniach oraz poznanie procesu jego produkcji. Kolejne dni spędzisz na Kangaroo Island, gdzie spotkasz dzikie zwierzęta, w tym lwy morskie i oczywiście kangury, a także zachwycisz się spektakularnymi widokami skalnych formacji i pustych plaż.", 
     4, 5),
    
    (7, "Melbourne: Kultura, Kawa i Street Art", 
     "Melbourne to mekka dla miłośników kultury i dobrej kawy. W trakcie wycieczki przemierzysz najbardziej znane zakątki tego miasta. Spacer po ukrytych pasażach handlowych i ulicach ozdobionych kolorowym street artem pozwoli Ci odkryć artystyczną duszę Melbourne. Zajrzymy do kawiarni w Fitzroy, aby poznać lokalne tradycje picia kawy, a potem zwiedzimy Galerię Narodową Wiktorii oraz słynny stadion MCG. Nie zabraknie też wycieczki na pobliską Phillip Island, gdzie wieczorem zobaczysz paradę pingwinów wracających na brzeg. Na zakończenie spróbujesz lokalnych przysmaków w modnych restauracjach i odwiedzisz dzielnicę St Kilda.", 
     2, 6),
    
    (8, "Wombaty i inne cuda australijskiej przyrody", 
     "Rozpocznij swoją australijską przygodę w Melbourne, skąd wyruszysz na zwiedzanie najpiękniejszych parków narodowych południowo-wschodniej Australii. Podczas wyprawy odwiedzisz rezerwat Wilsons Promontory, gdzie spotkasz wombaty, kangury i emu w ich naturalnym środowisku. Na trasie znajdzie się również Park Narodowy Grampians – idealne miejsce na trekking i podziwianie wodospadów MacKenzie. W programie przewidziano warsztaty fotograficzne z lokalnym przewodnikiem.", 
     1, 6),
    
    (9, "Miasta i cuda natury", 
     "Kompleksowa wyprawa, która łączy największe miasta i naturalne atrakcje Australii. Zaczynasz w Sydney, gdzie zwiedzisz słynny Opera House i pospacerujesz po plaży Bondi. Następnie udasz się do Melbourne, aby poznać jego artystyczną duszę. W planie jest również przejazd Great Ocean Road, czyli jednej z najpiękniejszych tras widokowych na świecie, z przystankiem przy Dwunastu Apostołach. W programie także trekking w Blue Mountains oraz relaks na złotych plażach Gold Coast.", 
     1, 3),
    
    (10, "Przygoda w Brisbane", 
     "Brisbane, z jego subtropikalnym klimatem i tętniącym życiem centrum, oferuje idealne połączenie relaksu i aktywności. W tygodniowym programie znajdziesz zwiedzanie centrum miasta, rejs po rzece Brisbane i wjazd na Mount Coot-tha, gdzie zachwyci Cię widok na panoramę miasta. Po drodze odwiedzisz Lone Pine Koala Sanctuary, gdzie przytulisz koalę i nakarmisz kangury. Czas wolny spędzisz w South Bank, tętniącym życiem miejscu nad rzeką, pełnym restauracji, barów i sztucznej miejskiej plaży. Na zakończenie wyruszysz na całodniową wycieczkę na Gold Coast, gdzie plaże i surfing stanowią kwintesencję australijskiego życia.", 
     3, 7)
]

cursor.executemany(insert_rodzaje_query, wycieczki)

connection.commit()


In [9]:
# Wypełnianie tabeli Koszty_organizacji

def generuj_nocleg():
    return round(random.uniform(200, 500), 2)

def generuj_przewodnik():
    return round(random.uniform(600, 2000), 2)

def generuj_loty():
    return round(random.uniform(5000, 8000), 2)

def generuj_atrakcje():
    return round(random.uniform(1000, 2500), 2)

def generuj_ubezpieczenie():
    return round(random.uniform(200, 300), 2)

def generuj_transport():
    return round(random.uniform(500, 1000), 2)

for _ in range(300):
    koszty = {
        "Nocleg_za_noc": generuj_nocleg(),
        "Przewodnik": generuj_przewodnik(),
        "Loty": generuj_loty(),
        "Atrakcje": generuj_atrakcje(),
        "Ubezpieczenie": generuj_ubezpieczenie(),
        "Transport": generuj_transport()
    }

    cursor.execute('''
        INSERT INTO Koszty_organizacji (Nocleg_za_noc, Przewodnik, Loty, Atrakcje, Ubezpieczenie, Transport)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''', (
        koszty["Nocleg_za_noc"],
        koszty["Przewodnik"],
        koszty["Loty"],
        koszty["Atrakcje"],
        koszty["Ubezpieczenie"],
        koszty["Transport"]
    ))

connection.commit()

In [10]:
# Wypełnianie tabeli Realizowane_wycieczki

cursor.execute("SELECT id_koszty, Nocleg_za_noc, Przewodnik, Loty, Atrakcje, Ubezpieczenie, Transport FROM Koszty_organizacji;")
koszty_organizacji = cursor.fetchall()
ilosc_id_koszty = len(koszty_organizacji)

if ilosc_id_koszty < 1:
    print("Błąd: brak dostępnych kosztów w tabeli Koszty_organizacji!")
    exit()

insert_realizowane_query = """
INSERT INTO Realizowane_wycieczki (id_wycieczki, id_rodzaj, cena_za_osobe, data_rozpoczecia, data_zakonczenia, id_koszty) 
VALUES (%s, %s, %s, %s, %s, %s);
"""

rodzaje_wycieczek = [
    {"id_rodzaj": 1, "dni_zakres": (10, 12)},
    {"id_rodzaj": 2, "dni_zakres": (10, 12)},
    {"id_rodzaj": 3, "dni_zakres": (10, 12)},
    {"id_rodzaj": 4, "dni_zakres": (14, 16)},
    {"id_rodzaj": 5, "dni_zakres": (14, 16)},
    {"id_rodzaj": 6, "dni_zakres": (14, 16)},
    {"id_rodzaj": 7, "dni_zakres": (14, 16)},
    {"id_rodzaj": 8, "dni_zakres": (18, 20)},
    {"id_rodzaj": 9, "dni_zakres": (18, 20)},
    {"id_rodzaj": 10, "dni_zakres": (18, 20)}
]

# Funkcja do generowania daty wyjazdu
def losuj_date_wyjazdu():
    start_date = datetime(2022, 3, 1)
    end_date = datetime(2025, 3, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

random.shuffle(koszty_organizacji)


for i, koszty in enumerate(koszty_organizacji):
    id_koszty = koszty[0]
    nocleg_za_noc = koszty[1]
    przewodnik = koszty[2]
    loty = koszty[3]
    atrakcje = koszty[4]
    ubezpieczenie = koszty[5]
    transport = koszty[6]

    rodzaj = random.choice(rodzaje_wycieczek)
    ilosc_nocy = random.randint(rodzaj["dni_zakres"][0], rodzaj["dni_zakres"][1]) - 1
    data_wyjazdu = losuj_date_wyjazdu()
    data_przyjazdu = data_wyjazdu + timedelta(days=ilosc_nocy + 2)

    # Obliczanie ceny za osobę
    cena_za_osobe = (
    (nocleg_za_noc * ilosc_nocy) +
    przewodnik +
    loty +
    atrakcje +
    ubezpieczenie +
    transport
)

    # Losowy procent zwiększenia od 10% do 30%
    procent_zwiekszenia = Decimal(random.uniform(0.1, 0.3))
    cena_za_osobe *= (Decimal(1) + procent_zwiekszenia)

    # Zaokrąglanie do najbliższej liczby całkowitej
    cena_za_osobe = cena_za_osobe.quantize(Decimal('1'), rounding=ROUND_HALF_UP)

    cursor.execute(insert_realizowane_query, (i + 1, rodzaj["id_rodzaj"], cena_za_osobe,
                                       data_wyjazdu.strftime('%Y-%m-%d'), data_przyjazdu.strftime('%Y-%m-%d'), id_koszty))

connection.commit()


In [11]:
# Wypełnianie tabeli Metody_platnosci

insert_metody_query = """INSERT INTO Metody_platnosci (Metoda) 
VALUES 
('Przelew'),
('Blik'),
('Płatność kartą'),
('Apple Pay'),
('Google Pay');
"""
cursor.execute(insert_metody_query)
connection.commit()

In [12]:
# Wypełnianie tabeli Transakcje_finansowe

def generuj_zrealizowano():
    return 1 if random.random() < random.uniform(0.93, 0.98) else 0

def generuj_id_metody():
    wybor = random.random()
    if wybor < random.uniform(0.55, 0.65): 
        return random.choice([1, 2])  
    elif wybor < random.uniform(0.70, 0.90): 
        return 3  
    else:  
        return random.choice([4, 5])  

for _ in range(3500):  
    zrealizowano = generuj_zrealizowano()
    id_metody = generuj_id_metody()

    cursor.execute(""" 
        INSERT INTO Transakcje_finansowe (Zrealizowano, id_metody)
        VALUES (%s, %s)
    """, (zrealizowano, id_metody))

connection.commit()

In [13]:
# Wypełnianie tabeli Rezerwacje

cursor.execute("SELECT COALESCE(MAX(id_rezerwacje), 0) FROM Rezerwacje")
max_id_rezerwacje = cursor.fetchone()[0]

insert_rezerwacje_query = """
INSERT INTO Rezerwacje (id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, Data_rezerwacji)
VALUES (%s, %s, %s, %s, %s, %s);
"""

dzien_startu = datetime(2022, 1, 14).date()
dzis = datetime.today().date()

def losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia):
    max_date = min(data_rozpoczecia - timedelta(days=2), dzis)
    min_date = max(dzien_startu, data_urodzenia + timedelta(weeks=52*18), data_rozpoczecia - timedelta(weeks=26))
    if min_date > max_date:
        min_date = max_date  
    delta = max_date - min_date
    random_days = random.randint(0, delta.days)
    return min_date + timedelta(days=random_days)

cursor.execute("SELECT id_wycieczki, Data_rozpoczecia FROM Realizowane_wycieczki")
wycieczki = cursor.fetchall()

cursor.execute("SELECT id_klienci, Data_urodzenia FROM Klienci")
klienci = cursor.fetchall()

cursor.execute("SELECT id_pracownicy FROM Pracownicy")
pracownicy = cursor.fetchall()

cursor.execute("SELECT id_transakcje FROM Transakcje_finansowe")
transakcje = [transakcja[0] for transakcja in cursor.fetchall()]

random.shuffle(transakcje)
random.shuffle(klienci)

rezerwacje = []

for klient, data_urodzenia in klienci:
    id_klienci = klient
    while True:
        id_wycieczki, data_rozpoczecia = random.choice(wycieczki)
        data_rezerwacji = losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia)
        wiek_klienta = (data_rezerwacji - data_urodzenia).days // 365
        if wiek_klienta >= 18:
            break
    id_pracownicy = random.choice([5, 6, 7])
    max_id_rezerwacje += 1
    rezerwacje.append((max_id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, None, data_rezerwacji.strftime('%Y-%m-%d')))

while len(rezerwacje) < 3500:
    id_klienci, data_urodzenia = random.choice(klienci)
    id_wycieczki, data_rozpoczecia = random.choice(wycieczki)
    data_rezerwacji = losuj_date_rezerwacji(data_rozpoczecia, data_urodzenia)
    wiek_klienta = (data_rezerwacji - data_urodzenia).days // 365
    if wiek_klienta < 18:
        continue
    id_pracownicy = random.choice([5, 6, 7])
    max_id_rezerwacje += 1
    rezerwacje.append((max_id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, None, data_rezerwacji.strftime('%Y-%m-%d')))
for i in range(len(rezerwacje)):
    rezerwacje[i] = (rezerwacje[i][0], *rezerwacje[i][1:4], transakcje[i], rezerwacje[i][5])

cursor.executemany(insert_rezerwacje_query, rezerwacje)
connection.commit()

In [14]:
# Wypełnianie tabeli oceny

insert_oceny_query = """
INSERT INTO Oceny (id_rezerwacje, Ocena, Data_oceny) 
VALUES (%s, %s, %s);
"""

cursor.execute("""
    SELECT 
        r.id_rezerwacje, 
        rw.data_rozpoczecia, 
        rw.data_zakonczenia
    FROM Rezerwacje r
    INNER JOIN Transakcje_finansowe tf ON r.id_transakcje = tf.id_transakcje
    INNER JOIN Realizowane_wycieczki rw ON r.id_wycieczki = rw.id_wycieczki
    WHERE tf.Zrealizowano = 1
""")

rezerwacje = cursor.fetchall()

cursor.execute("SELECT id_rezerwacje FROM Oceny")
ocenione_rezerwacje = {row[0] for row in cursor.fetchall()}
nieocenione_rezerwacje = [rez for rez in rezerwacje if rez[0] not in ocenione_rezerwacje]
liczba_rezerwacji = len(nieocenione_rezerwacje)
liczba_do_oceny = random.randint(int(liczba_rezerwacji * 0.7), int(liczba_rezerwacji * 0.8))
rezerwacje_do_oceny = random.sample(nieocenione_rezerwacje, liczba_do_oceny)

for (id_rezerwacje, data_rozpoczecia, data_zakonczenia) in rezerwacje_do_oceny:
    ocena = random.randint(1, 10) 
    data_oceny = data_zakonczenia + timedelta(days=random.randint(1, 30)) 
    if data_oceny <= date.today():
        cursor.execute(insert_oceny_query, (id_rezerwacje, ocena, data_oceny.strftime('%Y-%m-%d')))
        
connection.commit()


In [15]:
cursor.close()
connection.close()